In [3]:
import pandas as pd
df =pd.read_csv(r"C:\Users\Shabaan\Desktop\V\testemails.csv")

In [4]:
df.head()

,file,message,TEXT_CONTENT
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [5]:
import email

def get_text_content(msg):
    text_content = ""
    
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get('Content-Disposition'))

            # Skip any non-text parts
            if 'text' in content_type and 'attachment' not in content_disposition:
                text_content += part.get_payload(decode=True).decode(part.get_content_charset() or 'utf-8', errors='ignore') + '\n'
    else:
        text_content = msg.get_payload(decode=True).decode(msg.get_content_charset() or 'utf-8', errors='ignore')

    return text_content.strip()

# Create a new column 'text_data' in the DataFrame
df['TEXT_CONTENT'] = ""

# Iterate through each row in the DataFrame and extract text content
for index, row in df.iterrows():
    # Assuming 'email_content' is the column containing email content in your DataFrame
    email_content = row['message']

    # Parse the email using email.message_from_string
    msg = email.message_from_string(email_content)

    # Extract text content from the email
    email_text_content = get_text_content(msg)

    # Assign the text content to the 'text_data' column
    df.at[index, 'TEXT_CONTENT'] = email_text_content

# Save the updated DataFrame to a new CSV file or overwrite the existing file
df.to_csv('testemails.csv', index=False)

In [6]:
df.head()

,file,message,TEXT_CONTENT
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [7]:
import nltk

# Download the punkt tokenizer data
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


# Extract the email body
body = df['TEXT_CONTENT']
df['StemmedContent'] = ""
for i in range(0,len(body)):
    # Tokenize the text
    tokens = word_tokenize(body[i])

    # Remove stop words and punctuation
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]

    # Stemming
    ps = PorterStemmer()
    stemmed_tokens = [ps.stem(word) for word in filtered_tokens]
    df.at[i, 'StemmedContent'] = stemmed_tokens


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shabaan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shabaan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df.tail()

,file,message,TEXT_CONTENT,StemmedContent
495,allen-p/_sent_mail/549.,Message-ID: <22854556.1075855728255.JavaMail.e...,"Patti,\n\nThis sounds like an opportunity to l...","[patti, sound, like, opportun, land, coupl, an..."
496,allen-p/_sent_mail/55.,Message-ID: <25123422.1075855686459.JavaMail.e...,I would support Matt Lenhart's promotion to th...,"[would, support, matt, lenhart, promot, next, ..."
497,allen-p/_sent_mail/550.,Message-ID: <18949220.1075855728276.JavaMail.e...,"Nick,\n\nThere is a specific program that we a...","[nick, specif, program, use, recruit, train, m..."
498,allen-p/_sent_mail/551.,Message-ID: <14439969.1075855728298.JavaMail.e...,---------------------- Forwarded by Phillip K ...,"[forward, phillip, k, pm, enron, north, americ..."
499,allen-p/_sent_mail/552.,Message-ID: <26913174.1075855728319.JavaMail.e...,"Lucy,\n\n#32 and #29 are fine. \n\n#28 paid w...","[luci, 32, 29, fine, 28, paid, weekli, switch,..."


In [9]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

from nltk import word_tokenize, pos_tag, ne_chunk

# Sample text
body = df['TEXT_CONTENT']
df['EntitytText']=""
df['EntityLabel']=""

# Tokenize and part-of-speech tagging
for i in range(0,len(body)):
    tokens = word_tokenize(body[i])
    pos_tags = pos_tag(tokens)

    # Named Entity Recognition using ne_chunk
    named_entities = ne_chunk(pos_tags)

    # Display named entities
    for chunk in named_entities:
        if isinstance(chunk, nltk.Tree):
            entity_label = chunk.label()
            df.at[i,'EntityLabel']=entity_label
            entity_text = " ".join([token[0] for token in chunk.leaves()])
            df.at[i, 'EntityText'] = entity_text
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Shabaan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Shabaan\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Shabaan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


,file,message,TEXT_CONTENT,StemmedContent,EntitytText,EntityLabel,EntityText
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast,[forecast],,,nan
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,"[travel, busi, meet, take, fun, trip, especi, ...",,PERSON,Austin
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,"[test, success, way, go]",,,nan
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...","[randi, send, schedul, salari, level, everyon,...",,PERSON,Phillip
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,"[let, shoot, tuesday]",,,nan
...,...,...,...,...,...,...,...
495,allen-p/_sent_mail/549.,Message-ID: <22854556.1075855728255.JavaMail.e...,"Patti,\n\nThis sounds like an opportunity to l...","[patti, sound, like, opportun, land, coupl, an...",,PERSON,Jana
496,allen-p/_sent_mail/55.,Message-ID: <25123422.1075855686459.JavaMail.e...,I would support Matt Lenhart's promotion to th...,"[would, support, matt, lenhart, promot, next, ...",,PERSON,Ken Shulklapper
497,allen-p/_sent_mail/550.,Message-ID: <18949220.1075855728276.JavaMail.e...,"Nick,\n\nThere is a specific program that we a...","[nick, specif, program, use, recruit, train, m...",,PERSON,Ted Bland
498,allen-p/_sent_mail/551.,Message-ID: <14439969.1075855728298.JavaMail.e...,---------------------- Forwarded by Phillip K ...,"[forward, phillip, k, pm, enron, north, americ...",,PERSON,Kidventure Camp


In [10]:
df=df.dropna()

In [11]:
df.head()

,file,message,TEXT_CONTENT,StemmedContent,EntitytText,EntityLabel,EntityText
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast,[forecast],,,nan
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,"[travel, busi, meet, take, fun, trip, especi, ...",,PERSON,Austin
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,"[test, success, way, go]",,,nan
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...","[randi, send, schedul, salari, level, everyon,...",,PERSON,Phillip
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,"[let, shoot, tuesday]",,,nan


In [12]:
df=df.dropna()

In [13]:
df

,file,message,TEXT_CONTENT,StemmedContent,EntitytText,EntityLabel,EntityText
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast,[forecast],,,nan
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,"[travel, busi, meet, take, fun, trip, especi, ...",,PERSON,Austin
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,"[test, success, way, go]",,,nan
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...","[randi, send, schedul, salari, level, everyon,...",,PERSON,Phillip
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,"[let, shoot, tuesday]",,,nan
...,...,...,...,...,...,...,...
495,allen-p/_sent_mail/549.,Message-ID: <22854556.1075855728255.JavaMail.e...,"Patti,\n\nThis sounds like an opportunity to l...","[patti, sound, like, opportun, land, coupl, an...",,PERSON,Jana
496,allen-p/_sent_mail/55.,Message-ID: <25123422.1075855686459.JavaMail.e...,I would support Matt Lenhart's promotion to th...,"[would, support, matt, lenhart, promot, next, ...",,PERSON,Ken Shulklapper
497,allen-p/_sent_mail/550.,Message-ID: <18949220.1075855728276.JavaMail.e...,"Nick,\n\nThere is a specific program that we a...","[nick, specif, program, use, recruit, train, m...",,PERSON,Ted Bland
498,allen-p/_sent_mail/551.,Message-ID: <14439969.1075855728298.JavaMail.e...,---------------------- Forwarded by Phillip K ...,"[forward, phillip, k, pm, enron, north, americ...",,PERSON,Kidventure Camp


In [14]:
# Assuming df is your DataFrame
df.to_csv('output.csv', index=False)
